In [0]:
%run /Workspace/Users/sireeshabyreddy96@gmail.com/real-time-weather-pipeline/Medalian_notebooks/Slack_utils

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit, monotonically_increasing_id
import uuid
import datetime

# ----------------------------
# Config
# ----------------------------
bronze_table = "weather_catalog.raw.weather_bronze"
silver_table = "weather_catalog.processed.valid_readings"
dlq_table = "weather_catalog.processed.dlq_silver"
log_table = "weather_catalog.logging.silver_ingestion_logs"

batch_id = str(uuid.uuid4())


# ----------------------------
# Step 1: Read Bronze Data
# ----------------------------
df_bronze = spark.table(bronze_table)


df_bronze.display(5, truncate=False)
df_bronze.printSchema()

send_slack_message(
    f" Bronze data loaded | Batch: {batch_id} | Rows: {df_bronze.count()}"
)


In [0]:
from pyspark.sql.functions import col, to_timestamp

# Step 1: Cast columns
df_casted = df_bronze.select(
    to_timestamp(col("date_time"), "yyyy-MM-dd HH:mm:ss").alias("date_time"),
    col("maxtempC").cast("double"),
    col("mintempC").cast("double"),
    col("totalSnow_cm").cast("double"),
    col("sunHour").cast("double"),
    col("uvIndex").cast("int"),
    col("moon_illumination").cast("int"),
    col("moonrise").cast("string"),
    col("moonset").cast("string"),
    col("sunrise").cast("string"),
    col("sunset").cast("string"),
    col("DewPointC").cast("int"),
    col("FeelsLikeC").cast("int"),
    col("HeatIndexC").cast("int"),
    col("WindChillC").cast("int"),
    col("WindGustKmph").cast("int"),
    col("cloudcover").cast("int"),
    col("humidity").cast("int"),
    col("precipMM").cast("double"),
    col("pressure").cast("int"),
    col("tempC").cast("int"),
    col("visibility").cast("int"),
    col("winddirDegree").cast("int"),
    col("windspeedKmph").cast("int"),
    col("City").cast("string")
)

# Step 2: Schema info
df_casted.printSchema()
send_slack_message("Schema casted successfully for Bronze data.")

# Step 3: Show a sample
df_casted.display(5, truncate=False)
send_slack_message(f"Sample data displayed. Row count: {df_casted.count()}")


In [0]:
from pyspark.sql.functions import col, sum, when

# Step 1: Count nulls per column
null_counts = df_casted.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_casted.columns])
null_counts.display()

# Step 2: Collect null counts as a dictionary
null_counts_dict = null_counts.collect()[0].asDict()

# Step 3: Format message for Slack
message = "Null count per column:\n"
for col_name, count in null_counts_dict.items():
    message += f"{col_name}: {count}\n"

# Step 4: Send to Slack
send_slack_message(message)


In [0]:
# Drop rows where more than 50% of columns are null
threshold = int(len(df_casted.columns) * 0.5)
df_cleaned = df_casted.dropna(thresh=threshold)

# Count rows before and after
rows_before = df_casted.count()
rows_after = df_cleaned.count()
rows_dropped = rows_before - rows_after

# Display sample
df_cleaned.display(5, truncate=False)

# Send Slack notification
message = (f"Dropped rows with >50% nulls\n"
           f"Rows before: {rows_before}\n"
           f"Rows after: {rows_after}\n"
           f"Rows dropped: {rows_dropped}")
send_slack_message(message)
df_cleaned.show()
df_cleaned.count()

In [0]:
from pyspark.sql.functions import col, sum, when


#  Remove rows with 0 in critical numeric columns
critical_cols = ["maxtempC", "mintempC", "humidity", "pressure", "windspeedKmph"]
for c in critical_cols:
    df_casted = df_casted.filter(col(c) != 0)
send_slack_message(f" Removed rows with 0 in critical columns: {', '.join(critical_cols)}")

#  Remove rows with empty/UNKNOWN in string columns
string_cols = [f.name for f in df_casted.schema.fields if str(f.dataType) == "StringType"]
for c in string_cols:
    df_casted = df_casted.filter((col(c) != "UNKNOWN") & (col(c) != ""))
send_slack_message(f" Removed rows with empty/UNKNOWN in string columns: {', '.join(string_cols)}")

#  Check remaining nulls
null_counts = df_casted.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_casted.columns])
display(null_counts)

#  Count rows after cleaning
rows_after_cleaning = df_casted.count()
send_slack_message(f" Row count after cleaning: {rows_after_cleaning}")


In [0]:
from pyspark.sql.functions import col, sum, when

# Replace NULL values with 0 for numeric columns
df_filled = df_casted.fillna(0)

# Verify null counts again
null_check = df_filled.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_filled.columns])
display(null_check)


# Send Slack notification
message = (f" Nulls filled with 0 for numeric columns\n"
           f"Rows before filling: {rows_before}\n"
           f"Rows after filling: {rows_after}")
send_slack_message(message)


In [0]:
from pyspark.sql.functions import col, sum, when

# Count nulls per column, keep original types intact
null_counts = df_filled.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_filled.columns])
null_counts.display()


In [0]:
from pyspark.sql import functions as F

# Define columns with potential outliers to remove or clean
outlier_cols = [
    "maxtempC", "mintempC", "totalSnow_cm", "sunHour", "uvIndex",
    "DewPointC", "FeelsLikeC", "HeatIndexC", "WindChillC", "WindGustKmph",
    "cloudcover", "humidity", "precipMM", "pressure", "tempC", "visibility",
    "winddirDegree", "windspeedKmph"
]

# Filter out rows with invalid values (-999 or null) in any of these columns
for col in outlier_cols:
    df_filled = df_filled.filter((F.col(col).isNotNull()) & (F.col(col) != -999))

# Count duplicates after removing outliers
duplicate_count = df_filled.count() - df_filled.dropDuplicates().count()
print(f"Total duplicate rows: {duplicate_count}")

# Optional: Show top 10 duplicate rows
df_filled.groupBy(df_filled.columns).count().filter("count > 1").display(10, truncate=False)

# Remove duplicates
df_no_duplicates = df_filled.dropDuplicates()

# Send Slack notification
if duplicate_count > 0:
    send_slack_message(f"Removed {duplicate_count} duplicate rows.")
else:
    send_slack_message("No duplicate rows found.")

# df_no_duplicates is now your cleaned DataFrame without duplicates or outliers
df_no_duplicates.count()


In [0]:
from pyspark.sql.functions import col

# Function to remove outliers using IQR for all numeric columns
def remove_outliers(df, numeric_cols):
    for c in numeric_cols:
        # Calculate Q1 and Q3
        q1, q3 = df.approxQuantile(c, [0.25, 0.75], 0.05)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Filter rows within bounds
        df = df.filter((col(c) >= lower_bound) & (col(c) <= upper_bound))
    
    return df

# Select numeric columns (exclude string columns like 'City')
numeric_cols = [field.name for field in df_no_duplicates.schema.fields if str(field.dataType) in ["IntegerType", "LongType", "FloatType", "DoubleType"]]

# Row count before outlier removal
row_count_before = df_no_duplicates.count()

# Remove outliers
df_no_outliers = remove_outliers(df_no_duplicates, numeric_cols)

# Row count after outlier removal
row_count_after = df_no_outliers.count()

# Display results
df_no_outliers.display()

# Slack notification
rows_removed = row_count_before - row_count_after
if rows_removed > 0:
    send_slack_message(
        f" Outlier removal completed. Rows removed: {rows_removed} | Remaining rows: {row_count_after}", 
        level="WARNING"
    )
else:
    send_slack_message(
        f" Outlier removal completed. No rows removed. Total rows: {row_count_after}"
       
    )

print("Row count after outlier removal:", row_count_after)


In [0]:
# ----------------------------
# Step 2: Enrich with Metadata
# ----------------------------
df_enriched = df_no_outliers.withColumn("ingest_time", current_timestamp()) \
                       .withColumn("batch_id", lit(batch_id)) \
                       .withColumn("record_id", monotonically_increasing_id())

df_enriched.display(5)
df_enriched.count()


In [0]:
# ----------------------------

# Remove rows with null city or date_time
# ----------------------------
df_dlq = df_enriched.filter(
    col("city").isNull() | col("date_time").isNull()
)

dlq_count = df_dlq.count()
print(f"Rows sent to DLQ: {dlq_count}")

# Slack notification
if dlq_count > 0:
    send_slack_message(
        f"Validation detected {dlq_count} invalid rows sent to DLQ.",
        level="WARNING"
    )
else:
    send_slack_message(
        " Validation completed. No rows sent to DLQ."
    )


In [0]:
# Ensure the logging schema exists
spark.sql("CREATE SCHEMA IF NOT EXISTS weather_catalog.logging")

# Step 4: Write Silver Data
df_enriched.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("weather_catalog.raw.silver_table")

# Step 5: Write DLQ Data
dlq_count = df_dlq.count()
if dlq_count > 0:
    df_dlq.write.format("delta") \
        .mode("append") \
        .saveAsTable("weather_catalog.logging .dlq_table")

# Step 6: Slack Notification
silver_count = df_enriched.count()
message = f" Weather ETL completed.\n- Silver Records: {silver_count}\n- DLQ Records: {dlq_count}"
send_slack_message(message)
print(message)
